# Map eigenvalues

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"../..") # Allow import of agd from parent directory (useless if conda package installed)
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Riemannian','FMM'))

In [2]:
from agd import HFMUtils
from agd.Metrics import Riemann
from agd import LinearParallel as lp
from agd import FiniteDifferences as fd
from agd.Plotting import savefig; #savefig.dirName = 'Figures/Riemannian'
from agd.AutomaticDifferentiation.Optimization import norm_infinity

In [3]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

# Required for 3D plotting
from mpl_toolkits.mplot3d import Axes3D # Plots geodesics
useMayavi = False
if useMayavi:
    from mayavi import mlab # Plots implicit surfaces

## 1. Testing

In [4]:
M = np.eye(2).reshape((1,2,2))

In [5]:
M = np.random.normal(size=(3,3,2,4))
M = M+lp.transpose(M)

In [6]:
M_ = np.moveaxis(M,(0,1),(-2,-1))
ev_,vp_ = np.linalg.eig(M_)
ev = np.moveaxis(ev_,-1,0)
vp = np.moveaxis(vp_,(-2,-1),(0,1))

In [7]:
print(f"{ev.shape}, {vp.shape}")

(3, 2, 4), (3, 3, 2, 4)


In [8]:
outer = lp.outer(vp,ev*vp)
norm_infinity(M - outer.sum(axis=2))

3.9968028886505635e-15

In [9]:
outer = lp.outer(vp,(1/ev)*vp)
MInv = outer.sum(axis=2)
Id = fd.as_field(np.eye(3),M.shape[2:])
norm_infinity(Id - lp.dot_AA(M,MInv))

6.394884621840902e-14

In [10]:
order = ev.argsort(axis=0)
ev_sorted = np.take_along_axis(ev,order,axis=0)

In [11]:
ev_sorted[:,0,0]

array([-4.10198405, -2.81974936,  1.28721789])

In [12]:
order_inv = order.argsort(axis=0)
ev_orig = np.take_along_axis(ev_sorted,order_inv,axis=0)

In [13]:
(ev-ev_orig).shape

(3, 2, 4)

In [14]:
riem = Riemann.from_mapped_eigenvalues(M,lambda x:1./x)

In [15]:
norm_infinity(riem.m - lp.inverse(M))

2.8421709430404007e-13

In [19]:
def f(x):
    x0,x1,x2 = x
    assert np.all(x0<=x1) and np.all(x1<=x2)
    return x0,x1+1,x2+2
def g(x):
    x0,x1,x2 = x
    return x0,x1-1,x2-2

riem_f = Riemann.from_mapped_eigenvalues(M,f)
riem_orig = Riemann.from_mapped_eigenvalues(riem_f.m,g)
norm_infinity(riem_orig.m-M)

5.551115123125783e-15